## Generate SVG's for to plot a function on a grid

## IMPORTANT: 
Unfortunately this only works with v0.6 at the moment. The translation of iterators from 0.6 to 0.7+ is not exactly straight forward, at least for something funny like `ClosedCurve` which has cyclic indices.

In [1]:
include("glowify_func.jl")

glowify_grid_func (generic function with 2 methods)

We'll use two coordinate systems - one concrete (for the material), one abstract (for the mathematical). First we make a function and specify the grid on which to sample it.

In [2]:
# 2D function to sample
grid_func(x,y) = y.^2 - x.^2


# grid along which to sample the function:
#DiscreteDomain(Nx::Int64, Ny::Int64, minmin::PointsNCurves.Point, maxmax::PointsNCurves.Point) 
#Nx = (number of planks along the x direction) + 2
#Ny = (number of planks along the y direction) + 2
#minmin = Point(min_x, min_y), "lower left" corner of sampling domain
#maxmax = Point(max_x, max_y), "upper right" corner of sampling domain
disc_dom = DiscreteDomain(8,8,Point(-2,-2),Point(2,2))

# This will make 6 planks!

DiscreteDomain(8, 8, PointsNCurves.Point(-2.0, -2.0), PointsNCurves.Point(2.0, 2.0))

In [3]:
# Now produce two sets of curves representing the cuts in both directions:
# xplanks, yplanks 
#        = glowify_grid_func(
#               grid_func::Function,
#               disc_dom::DiscreteDomain,
#               mat_width::Float64, <- In the material units
#               x_length::Float64,
#               y_length::Float64,
#               z_length::Float64, <- This is the amount to scale the function values by
#               z_min::Float64, <- The minimum depth of a cut. So total height is z_min + z_length
#               fs::Int=100 <- sample frequency per unit of length to sample the function
#               )
xplanks, yplanks = glowify_grid_func(grid_func, disc_dom, .164, 4., 4., 3., 1., 10);

In [5]:
# Finally plot draw the curves into an svg. They will all be drawn on top of each other. 
# TODO: draw them not on top of each other. 
# draw_glowified_fun(filename, x_planks, y_planks, unit)
draw_glowified_fun("hyperbolic.svg", xplanks, yplanks, "cm");

Drawing 12 curves!
drawing the svg to /Users/prc/Projects/ART/glowify_func/hyperbolic.svg
